# Reading JSON Files in Pandas

In this notebook, we will explore how to work with JSON files using Pandas. We will cover:
1. Reading JSON with `pd.read_json()`
2. Converting DataFrames to JSON with `df.to_json()`
3. Flattening nested JSON using `pd.json_normalize()`


## 1. Read JSON with `pd.read_json()`
### Example: JSON string with nested dictionary

In [1]:
import pandas as pd
from io import StringIO

data = '{"employee_name": "kashish", "email": "kashish.sachdeva.42@gmail.com", "job_profile": [{"title": "Team Lead", "title2": "Data scientist"}]}'

# Check type
print(type(data))  # str

# Read JSON

df = pd.read_json(StringIO(data))
print(df)


<class 'str'>
  employee_name                          email  \
0       kashish  kashish.sachdeva.42@gmail.com   

                                         job_profile  
0  {'title': 'Team Lead', 'title2': 'Data scienti...  


## 2. JSON Orient Formats
You can load JSON in different orientations:

In [8]:
df = pd.DataFrame([['a','b'], ['c','d']], index=['row1', 'row2'], columns=['col1', 'col2'])

# Different JSON orientations
print("Default:", df.to_json())
print("Index:", df.to_json(orient='index'))
print("Columns:", df.to_json(orient='columns'))
print("Records:", df.to_json(orient='records'))
print("Split:", df.to_json(orient='split'))
print("Table:", df.to_json(orient='table'))

Default: {"col1":{"row1":"a","row2":"c"},"col2":{"row1":"b","row2":"d"}}
Index: {"row1":{"col1":"a","col2":"b"},"row2":{"col1":"c","col2":"d"}}
Columns: {"col1":{"row1":"a","row2":"c"},"col2":{"row1":"b","row2":"d"}}
Records: [{"col1":"a","col2":"b"},{"col1":"c","col2":"d"}]
Split: {"columns":["col1","col2"],"index":["row1","row2"],"data":[["a","b"],["c","d"]]}
Table: {"schema":{"fields":[{"name":"index","type":"string"},{"name":"col1","type":"string"},{"name":"col2","type":"string"}],"primaryKey":["index"],"pandas_version":"1.4.0"},"data":[{"index":"row1","col1":"a","col2":"b"},{"index":"row2","col1":"c","col2":"d"}]}


In [10]:
# Read using 'table' orient
schema = df.to_json(orient='table')
pd.read_json(StringIO(schema), orient='table')

,col1,col2
row1,a,b
row2,c,d


## 3. Flatten Nested JSON using `pd.json_normalize()`
### Example 1: Nested dict inside list

In [11]:
data = [{
    "employee_name": "kashish",
    "email": "kashish.sachdeva.42@gmail.com",
    "job_profile": {
        "title": "Team Lead",
        "title2": "Data scientist"
    }
}]

pd.json_normalize(data)


,employee_name,email,job_profile.title,job_profile.title2
0,kashish,kashish.sachdeva.42@gmail.com,Team Lead,Data scientist


### Example 2: Nested inside list of dicts

In [12]:
data = [
    {"id": 1, "name": "Cole Volk", "fitness": {"height": 130, "weight": 60}},
    {"name": "Mark Reg", "fitness": {"height": 130, "weight": 60}},
    {"id": 2, "name": "Faye Raker", "fitness": {"height": 130, "weight": 60}}
]

# Default (no flattening)
print(pd.json_normalize(data))

# Flatten one level
print(pd.json_normalize(data, max_level=1))


    id        name  fitness.height  fitness.weight
0  1.0   Cole Volk             130              60
1  NaN    Mark Reg             130              60
2  2.0  Faye Raker             130              60
    id        name  fitness.height  fitness.weight
0  1.0   Cole Volk             130              60
1  NaN    Mark Reg             130              60
2  2.0  Faye Raker             130              60


## 4. Advanced Example: JSON with nested lists and dicts

In [13]:
data = [
    {
        "state": "Florida",
        "shortname": "FL",
        "info": {"governor": "Rick Scott"},
        "counties": [
            {"name": "Dade", "population": 12345},
            {"name": "Broward", "population": 40000},
            {"name": "Palm Beach", "population": 60000}
        ]
    },
    {
        "state": "Ohio",
        "shortname": "OH",
        "info": {"governor": "John Kasich"},
        "counties": [
            {"name": "Summit", "population": 1234},
            {"name": "Cuyahoga", "population": 1337}
        ]
    }
]

pd.json_normalize(
    data,
    record_path="counties",
    meta=["state", "shortname", ["info", "governor"]]
)


,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


## Summary

| Task                  | Function              |
|-----------------------|------------------------|
| Read JSON string      | `pd.read_json()`       |
| Convert to JSON       | `df.to_json()`         |
| Flatten nested JSON   | `pd.json_normalize()`  |
| Handle formats        | Use `orient=` carefully |
